In [4]:
from torchvision.models.detection.rpn import RegionProposalNetwork, AnchorGenerator, RPNHead
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.detection.roi_heads import roi_align
from torch.utils.data.dataloader import DataLoader
from models.backbone.ResNet import resnet12
from torchvision.models.detection.image_list import ImageList
from utils.dataset_tools.coco import read_coco_detection, show_dataset
from torchvision.transforms import transforms

# root : coco图片路径
# annFile : 标注文件路径
# transform : 图像转换(用于PIL)
# target_transform : 标注转换
# transforms : 图像和标注的转换

# 数据集预处理
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)])
t = transforms.Compose([transforms.ToTensor(),
                        transforms.Resize(600),  # 短边缩为600
                        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                        transforms.RandomHorizontalFlip(0.5)])

# 读取数据集
fsod_dataset = read_coco_detection(img_transform=t)

# support选取


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)])
transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Resize((600, 600)),
                                 transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                 transforms.RandomHorizontalFlip(0.5)])
fsod_dataset = read_coco_detection(img_transform=transforms)

# for i in range(len(fsod_dataset)):
#     img, labels = fsod_dataset.__getitem__(i)
#     print(img.shape)
#     print(len(labels))

fsod_dataloader = DataLoader(fsod_dataset, batch_size=5, shuffle=True, drop_last=True)

for img, labels in enumerate(fsod_dataloader):
    print(img)

loading annotations into memory...
Done (t=1.43s)
creating index...
index created!
loading annotations into memory...
Done (t=0.97s)
creating index...
index created!


RuntimeError: each element in list of batch should be of equal size

In [ ]:
# N-way K-shot:
# support: N个类别, 每个类别K个实例
# query: 不限, 只要在N个类别内即可

# 构建N类support, 每个类别的support含有k个实例

def generate_support():
    pass

In [ ]:
import math
import numpy as np
import cv2


def crop_support(self, roidb):
    # Get support box
    '''
    data_height, data_width = map(int, blobs['im_info'][:2])
    im_scale = blobs['im_info'][-1]
    #print(blobs['roidb'])
    print(blobs['im_info'])
    print('support data:', blobs['data'].shape)
    all_box = (blobs['roidb'][0]['boxes'] * im_scale).astype(np.int16)
    '''
    img_path = roidb[0]['image']
    all_box = roidb[0]['boxes']
    all_cls = np.array(roidb[0]['gt_classes'])
    target_cls = roidb[0]['target_cls']

    target_idx = np.where(all_cls == target_cls)[0]

    img = cv2.imread(img_path)
    if roidb[0]['flipped']:
        img = img[:, ::-1, :]
    img = img.astype(np.float32, copy=False)
    # img -= cfg.PIXEL_MEANS # np.array([[[102.9801, 115.9465, 122.7717]]])
    img -= np.array([[[102.9801, 115.9465, 122.7717]]])  # np.array([[[102.9801, 115.9465, 122.7717]]])
    img = img.transpose(2, 0, 1)
    data_height = int(img.shape[1])
    data_width = int(img.shape[2])

    all_box_num = all_box.shape[0]
    picked_box_id = np.random.choice(target_idx)  #random.choice(range(all_box_num))
    #print(all_cls, target_cls, target_idx, picked_box_id)
    picked_box = all_box[picked_box_id, :][np.newaxis, :].astype(np.int16)
    #print('1', blobs['roidb'][0]['boxes'], '2', roidb[0]['boxes'], '3', picked_box)
    '''
    original_img = cv2.imread(img_path)
    if roidb[0]['flipped']:
        original_img = original_img[:, ::-1, :]
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    self.vis_image(original_img, picked_box[0], img_path.split('/')[-1][:-4] + '_real.jpg', './test')
    '''
    '''
    picked_box = (picked_box / im_scale).astype(np.int16)
    data_height = int(data_height / im_scale)
    data_width = int(data_width / im_scale)
    blobs['data'] = cv2.resize(blobs['data'], (data_height, data_width))
    '''
    #print('3', picked_box)
    x1 = picked_box[0][0]
    y1 = picked_box[0][1]
    x2 = picked_box[0][2]
    y2 = picked_box[0][3]

    width = x2 - x1
    height = y2 - y1
    context_pixel = 16  #int(16 * im_scale)

    new_x1 = 0
    new_y1 = 0
    new_x2 = width
    new_y2 = height

    target_size = (320, 320)  #(384, 384)

    if width >= height:
        crop_x1 = x1 - context_pixel
        crop_x2 = x2 + context_pixel

        # New_x1 and new_x2 will change when crop context or overflow
        new_x1 = new_x1 + context_pixel
        new_x2 = new_x1 + width
        if crop_x1 < 0:
            new_x1 = new_x1 + crop_x1
            new_x2 = new_x1 + width
            crop_x1 = 0
        if crop_x2 > data_width:
            crop_x2 = data_width

        short_size = height
        long_size = crop_x2 - crop_x1
        y_center = int((y2 + y1) / 2)  #math.ceil((y2 + y1) / 2)
        crop_y1 = int(y_center - (long_size / 2))  #int(y_center - math.ceil(long_size / 2))
        crop_y2 = int(y_center + (long_size / 2))  #int(y_center + math.floor(long_size / 2))

        # New_y1 and new_y2 will change when crop context or overflow
        new_y1 = new_y1 + math.ceil((long_size - short_size) / 2)
        new_y2 = new_y1 + height
        if crop_y1 < 0:
            new_y1 = new_y1 + crop_y1
            new_y2 = new_y1 + height
            crop_y1 = 0
        if crop_y2 > data_height:
            crop_y2 = data_height

        crop_short_size = crop_y2 - crop_y1
        crop_long_size = crop_x2 - crop_x1
        square = np.zeros((3, crop_long_size, crop_long_size), dtype=np.float32)
        delta = int((crop_long_size - crop_short_size) / 2)  #int(math.ceil((crop_long_size - crop_short_size) / 2))
        square_y1 = delta
        square_y2 = delta + crop_short_size

        new_y1 = new_y1 + delta
        new_y2 = new_y2 + delta

        crop_box = img[:, crop_y1:crop_y2, crop_x1:crop_x2]

        square[:, square_y1:square_y2, :] = crop_box

        #show_square = np.zeros((crop_long_size, crop_long_size, 3))#, dtype=np.int16)
        #show_crop_box = original_img[crop_y1:crop_y2, crop_x1:crop_x2, :]
        #show_square[square_y1:square_y2, :, :] = show_crop_box
        #show_square = show_square.astype(np.int16)
    else:
        crop_y1 = y1 - context_pixel
        crop_y2 = y2 + context_pixel

        # New_y1 and new_y2 will change when crop context or overflow
        new_y1 = new_y1 + context_pixel
        new_y2 = new_y1 + height
        if crop_y1 < 0:
            new_y1 = new_y1 + crop_y1
            new_y2 = new_y1 + height
            crop_y1 = 0
        if crop_y2 > data_height:
            crop_y2 = data_height

        short_size = width
        long_size = crop_y2 - crop_y1
        x_center = int((x2 + x1) / 2)  #math.ceil((x2 + x1) / 2)
        crop_x1 = int(x_center - (long_size / 2))  #int(x_center - math.ceil(long_size / 2))
        crop_x2 = int(x_center + (long_size / 2))  #int(x_center + math.floor(long_size / 2))

        # New_x1 and new_x2 will change when crop context or overflow
        new_x1 = new_x1 + math.ceil((long_size - short_size) / 2)
        new_x2 = new_x1 + width
        if crop_x1 < 0:
            new_x1 = new_x1 + crop_x1
            new_x2 = new_x1 + width
            crop_x1 = 0
        if crop_x2 > data_width:
            crop_x2 = data_width

        crop_short_size = crop_x2 - crop_x1
        crop_long_size = crop_y2 - crop_y1
        square = np.zeros((3, crop_long_size, crop_long_size), dtype=np.float32)
        delta = int((crop_long_size - crop_short_size) / 2)  #int(math.ceil((crop_long_size - crop_short_size) / 2))
        square_x1 = delta
        square_x2 = delta + crop_short_size

        new_x1 = new_x1 + delta
        new_x2 = new_x2 + delta

        crop_box = img[:, crop_y1:crop_y2, crop_x1:crop_x2]
        square[:, :, square_x1:square_x2] = crop_box

        #show_square = np.zeros((crop_long_size, crop_long_size, 3)) #, dtype=np.int16)
        #show_crop_box = original_img[crop_y1:crop_y2, crop_x1:crop_x2, :]
        #show_square[:, square_x1:square_x2, :] = show_crop_box
        #show_square = show_square.astype(np.int16)

    square = square.astype(np.float32, copy=False)
    square_scale = float(target_size[0]) / long_size
    square = square.transpose(1, 2, 0)
    square = cv2.resize(square, target_size,
                        interpolation=cv2.INTER_LINEAR)  # None, None, fx=square_scale, fy=square_scale, interpolation=cv2.INTER_LINEAR)
    square = square.transpose(2, 0, 1)

    new_x1 = int(new_x1 * square_scale)
    new_y1 = int(new_y1 * square_scale)
    new_x2 = int(new_x2 * square_scale)
    new_y2 = int(new_y2 * square_scale)

    # For test
    #show_square = cv2.resize(show_square, target_size, interpolation=cv2.INTER_LINEAR) # None, None, fx=square_scale, fy=square_scale, interpolation=cv2.INTER_LINEAR)
    #self.vis_image(show_square, [new_x1, new_y1, new_x2, new_y2], img_path.split('/')[-1][:-4]+'_crop.jpg', './test')

    support_data = square
    support_box = np.array([[new_x1, new_y1, new_x2, new_y2]]).astype(np.float32)
    return support_data, support_box

In [19]:
import json
from torch.utils.data import Dataset
from utils.dataset_tools.coco import read_coco_detection
from PIL import Image


def class_dict(cocoDataset: Dataset):
    class_dicts = {}
    for img, labels in enumerate(cocoDataset):
        print(img, labels)
        for label in labels:
            class_index = label['category_id']
            if not class_index in class_dicts.keys():
                class_dicts[class_index] = []
                class_dicts[class_index].append(img.filename)
            else:
                if not img.filename in class_dicts[class_index]:
                    class_dicts[class_index].append(img.filename)
    json.dump(class_dicts, fp='class_dicts')



mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# normalize = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize(mean=mean, std=std)])
# transforms = transforms.Compose([transforms.ToTensor(),
#                                  transforms.Resize((600, 600)),
#                                  transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
#                                  transforms.RandomHorizontalFlip(0.5)])
fsod_dataset = read_coco_detection(img_transform=None)

# for i in range(len(fsod_dataset)):
#     img, labels = fsod_dataset.__getitem__(i)
#     print(img.shape)
#     print(len(labels))

class_dict(fsod_dataset)

img = Image.open('datasets/fsod/images/part_1/n00480993/n00480993_11037.jpg')
print(img, img.filename)

loading annotations into memory...
Done (t=1.86s)
creating index...
index created!
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=333x500 at 0x218889A5748> datasets/fsod/images/part_1/n00480993/n00480993_11037.jpg
